In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# ---------------------------------------------------------
# 1. 환경변수 로드 (notebooks 폴더의 상위 폴더에 있는 .env 로드)
# ---------------------------------------------------------
load_dotenv(dotenv_path="../.env")

# 2. DB 연결
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER", "admin"),
    "password": os.getenv("DB_PASSWORD"),
    "db": os.getenv("DB_NAME", "projectl")
}

db_url = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['db']}"
engine = create_engine(db_url)
print("✅ DB 연결 성공!")

# ---------------------------------------------------------
# 3. 데이터 수집 함수 (스키마 컬럼명 수정 완료)
# ---------------------------------------------------------
def get_item_data(item_name):
    conn = engine.connect()
    try:
        # (1) 아이템 ID 찾기
        item_sql = text("SELECT id FROM market_items WHERE name = :name")
        item_id = conn.execute(item_sql, {"name": item_name}).scalar()
        
        if not item_id:
            print(f"❌ '{item_name}' 아이템을 찾을 수 없습니다.")
            return None, None

        print(f"🔍 '{item_name}' (ID: {item_id}) 데이터 수집 중...")

        # (2) 가격 로그 가져오기 (컬럼명: current_min_price, logged_at)
        price_sql = text("""
            SELECT logged_at, current_min_price
            FROM market_price_logs
            WHERE item_id = :item_id
            ORDER BY logged_at ASC
        """)
        df_prices = pd.read_sql(price_sql, conn, params={"item_id": item_id})
        
        # (3) GPT Score 가져오기 (컬럼명: notice_date, gpt_score)
        notice_sql = text("""
            SELECT r.notice_date, i.gpt_score
            FROM item_notice_impacts i
            JOIN raw_notices r ON i.notice_id = r.id
            WHERE i.item_id = :item_id
            ORDER BY r.notice_date ASC
        """)
        df_notices = pd.read_sql(notice_sql, conn, params={"item_id": item_id})
        
        print(f"   - 가격 로그: {len(df_prices)}개")
        print(f"   - 관련 공지: {len(df_notices)}개")
        
        return df_prices, df_notices

    finally:
        conn.close()

# ---------------------------------------------------------
# [테스트] 아이템 이름 입력
# ---------------------------------------------------------
TARGET_ITEM = "유물 원한 각인서"  # 👈 분석하고 싶은 아이템 이름
df_raw, df_gpt = get_item_data(TARGET_ITEM)

display(df_raw.head())
display(df_gpt.head())

In [ ]:
import numpy as np
from datetime import timedelta

# =========================================================
# [업그레이드] 함수 1: 원본 청소 후 30분봉 만들기
# =========================================================
def preprocess_ohlc_and_fill(df_raw):
    print("🧹 [1단계] Raw 데이터 정제 및 30분봉 변환...")
    
    df = df_raw.copy()
    
    # 1. 인덱스 설정 (필수)
    if 'logged_at' in df.columns:
        df['logged_at'] = pd.to_datetime(df['logged_at'])
        df.set_index('logged_at', inplace=True)

    # -----------------------------------------------------
    # 🚨 [핵심 추가] 원본(10분 단위) 단계에서 이상치 1차 제거
    # -----------------------------------------------------
    # 10분 데이터이므로 하루(24시간)를 보려면 window가 약 144개 필요합니다.
    # (6개/시간 * 24시간 = 144)
    raw_window = 432
    raw_sigma = 7 # 원본은 변동성이 더 크니 2.0보다 살짝 여유 있게 2.5 추천
    
    # 원본 가격 컬럼: 'current_min_price'
    rolling_mean = df['current_min_price'].rolling(window=raw_window, center=True).mean()
    rolling_std = df['current_min_price'].rolling(window=raw_window, center=True).std()
    
    upper = rolling_mean + (raw_sigma * rolling_std)
    lower = rolling_mean - (raw_sigma * rolling_std)
    
    outliers = (df['current_min_price'] > upper) | (df['current_min_price'] < lower)
    cnt = outliers.sum()
    
    if cnt > 0:
        print(f"   ✨ 원본 데이터에서 {cnt}개의 이상치를 미리 제거했습니다!")
        df.loc[outliers, 'current_min_price'] = np.nan
        df['current_min_price'] = df['current_min_price'].interpolate(method='linear')
    # -----------------------------------------------------
    
    # 2. 30분 단위 Resampling (이제 깨끗한 재료로 요리합니다)
    df_resampled = df['current_min_price'].resample('30min').agg(['first', 'max', 'min', 'last', 'mean'])
    df_resampled.columns = ['Open', 'High', 'Low', 'Close', 'Price_Mean']
    
    # 3. 빈칸 채우기
    df_filled = df_resampled.ffill().bfill()
    
    print(f"   - 정제 완료: {len(df_filled)}개 구간 (30분봉)")
    return df_filled

# =========================================================
# [추가됨] 함수 1.5: 이상치 제거 (Rolling Z-Score)
# =========================================================
def clean_outliers_rolling(df, column='Close', window=48, sigma=3):
    print(f"🧼 [1.5단계] 이상치 제거 중... (Window={window}, Sigma={sigma})")
    df_clean = df.copy()
    
    # 이동 평균과 표준편차 계산
    rolling_mean = df_clean[column].rolling(window=window, center=True).mean()
    rolling_std = df_clean[column].rolling(window=window, center=True).std()
    
    # 이상치 판별 기준
    upper_bound = rolling_mean + (sigma * rolling_std)
    lower_bound = rolling_mean - (sigma * rolling_std)
    
    # 이상치 조건 확인 및 개수 카운트
    outlier_mask = (df_clean[column] > upper_bound) | (df_clean[column] < lower_bound)
    outlier_count = outlier_mask.sum()
    
    if outlier_count > 0:
        print(f"   🚨 {outlier_count}개의 튀는 가격 발견! 부드럽게 보정합니다.")
        # 이상치를 NaN으로 변경 후 보간(Interpolation)
        df_clean.loc[outlier_mask, column] = np.nan
        df_clean[column] = df_clean[column].interpolate(method='linear')
    else:
        print("   ✅ 데이터가 아주 깨끗합니다.")
        
    return df_clean

# =========================================================
# 함수 2: GPT 스코어 매핑 (날짜 계산 로직)
# =========================================================
def apply_gpt_scores(df_price, df_gpt):
    print("🤖 [2단계] GPT 공지사항 점수 매핑 중...")
    
    df = df_price.copy()
    df['GPT_Score'] = 0.0 # 기본값 0
    
    if df_gpt is None or df_gpt.empty:
        print("   - 공지사항 데이터가 없습니다. (점수 0으로 진행)")
        return df

    count = 0
    for idx, row in df_gpt.iterrows():
        notice_date = pd.to_datetime(row['notice_date'])
        score = row['gpt_score']
        
        # [규칙 적용] 공지일 10:00 ~ 7일 뒤 06:00
        start_time = notice_date.replace(hour=10, minute=0, second=0)
        end_time = (notice_date + timedelta(days=7)).replace(hour=6, minute=0, second=0)
        
        # 해당 기간 마스킹
        mask = (df.index >= start_time) & (df.index < end_time)
        
        if mask.any():
            df.loc[mask, 'GPT_Score'] = score
            count += 1
            
    print(f"   - 총 {count}건의 공지사항 점수 반영 완료")
    return df

# =========================================================
# [실행] 파이프라인 연결 (순서 중요!)
# =========================================================

# 1. 가격 데이터 1차 정제 (빈칸 채우기)
df_temp = preprocess_ohlc_and_fill(df_raw)

# 2. [추가] 이상치 제거 (여기서 튀는 값을 잡습니다!)
df_clean_price = clean_outliers_rolling(df_temp, column='Price_Mean', window=48, sigma=3)

# 3. GPT 점수 결합
df_final = apply_gpt_scores(df_clean_price, df_gpt)

# 4. 결과 확인
print("-" * 30)
print(f"🗓️ 데이터 기간: {df_final.index.min()} ~ {df_final.index.max()}")
print("-" * 30)
display(df_final.tail())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# ---------------------------------------------------------
# 3. 시각화 (확인용)
# ---------------------------------------------------------
plt.figure(figsize=(15, 6))

# 가격 그래프 (파란색)
ax1 = plt.gca()
ax1.plot(df_final.index, df_final['Price_Mean'], color='blue', label='Avg Price')
ax1.set_ylabel('Price (Gold)')

# GPT 점수 그래프 (빨간색, y축 공유)
ax2 = ax1.twinx()
ax2.plot(df_final.index, df_final['GPT_Score'], color='red', linestyle='--', alpha=0.5, label='GPT Score')
ax2.set_ylabel('GPT Score')
ax2.set_ylim(-1.5, 1.5)

# 2. 눈금 형식: '월-일' 형태로 보여줘라 (예: 01-29)
# 연도까지 보고 싶으면 '%Y-%m-%d'로 바꾸시면 됩니다.
ax1.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.WEDNESDAY, interval=1))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
ax1.grid(True, which='major', axis='x', linestyle='--', alpha=0.5) # 수요일마다 세로 점선 추가

plt.title(f"[{TARGET_ITEM}] Price Trend vs GPT Score")
plt.show()

# 데이터 확인
display(df_final.tail())

In [ ]:
import numpy as np

# 1. 원본 복사
df_ml = df_final.copy()

# ---------------------------------------------------------
# 2. 기술적 지표 (이동평균, 변동성, RSI)
# ---------------------------------------------------------
# 가격의 흐름(Trend)을 모델에게 알려줍니다.
df_ml['MA_5'] = df_ml['Close'].rolling(window=5).mean()    # 2.5시간 평균
df_ml['MA_48'] = df_ml['Close'].rolling(window=48).mean()  # 24시간(1일) 평균
df_ml['Std_20'] = df_ml['Close'].rolling(window=20).std()  # 변동성

# RSI (상대강도지수)
delta = df_ml['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df_ml['RSI'] = 100 - (100 / (1 + rs))

# ---------------------------------------------------------
# 3. [핵심] Lag Features (시차 데이터)
# ---------------------------------------------------------
# 모델 학습용으로 "과거의 정답"을 미리 만들어두는 겁니다.
# (나중에 예측할 때는 이 컬럼을 우리가 직접 채워넣으며 예측해야 합니다)
df_ml['Close_Lag1'] = df_ml['Close'].shift(1)  # 30분 전
df_ml['Close_Lag2'] = df_ml['Close'].shift(2)  # 1시간 전
df_ml['GPT_Lag1'] = df_ml['GPT_Score'].shift(1) # 30분 전 공지 점수

# ---------------------------------------------------------
# 4. 시간 정보 (Time Features)
# ---------------------------------------------------------
# 미래의 시간(요일, 시간)은 우리가 달력만 보면 100% 알 수 있으므로 아주 좋은 힌트입니다.
df_ml['Hour'] = df_ml.index.hour
df_ml['DayOfWeek'] = df_ml.index.dayofweek # 0:월 ~ 6:일

# ---------------------------------------------------------
# 5. Target (정답지: 다음 30분 뒤의 가격)
# ---------------------------------------------------------
df_ml['Target'] = df_ml['Close'].shift(-1)

# 결측치 제거 (이동평균 등으로 생긴 앞부분 빈칸 삭제)
df_ml = df_ml.dropna()

print(f"📚 머신러닝 데이터 준비 완료: {df_ml.shape}")
display(df_ml.tail())

In [ ]:
# ---------------------------------------------------------
# 1. 학습 데이터 준비 (규칙: 2월 4일 06:00 이전만 사용)
# ---------------------------------------------------------
# df_ml은 이미 정제 및 피처 엔지니어링이 끝난 데이터라고 가정합니다.
train_cutoff = pd.to_datetime("2026-02-04 06:00:00")

features = [
    'MA_5', 'MA_48', 'Std_20', 'RSI',
    'Close_Lag1', 'Close_Lag2', 'GPT_Lag1',
    'Hour', 'DayOfWeek'
]
target = 'Target'

# 학습용 데이터 (Cutoff 이전)
X_train = df_ml[df_ml.index < train_cutoff][features]
y_train = df_ml[df_ml.index < train_cutoff][target]

# 검증용 데이터 (Cutoff 이후 ~ 현재) - 과적합 방지용
X_valid = df_ml[df_ml.index >= train_cutoff][features]
y_valid = df_ml[df_ml.index >= train_cutoff][target]

print(f"📚 XGB 학습 데이터: {X_train.shape} ( ~ {train_cutoff})")
print(f"📝 XGB 검증 데이터: {X_valid.shape}")


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor

# ---------------------------------------------------------
# [필수] 평가 함수 다시 정의 (도구들이 import 된 상태에서)
# ---------------------------------------------------------
def get_detailed_score(y_true, y_pred):
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    # 1. 기본 오차
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    # 2. 퍼센트 오차 (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-7))) * 100
    
    # 3. 설명력 (R2)
    r2 = r2_score(y_true, y_pred)
    
    # 4. 방향 정확도
    true_diff = np.diff(y_true)
    pred_diff = np.diff(y_pred)
    correct_direction = np.sum(np.sign(true_diff) == np.sign(pred_diff))
    dir_acc = (correct_direction / len(true_diff)) * 100
    
    return rmse, mae, mape, r2, dir_acc

# ---------------------------------------------------------
# 1. 모델 정의 (LightGBM)
# ---------------------------------------------------------
print("⚡ LightGBM 학습 시작...")

model_lgbm = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=5,
    random_state=42,
    n_jobs=-1,
    force_col_wise=True,
    verbose=-1
)

# ---------------------------------------------------------
# 2. 학습
# ---------------------------------------------------------
model_lgbm.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric='rmse',
    callbacks=[]
)

# ---------------------------------------------------------
# 3. 평가
# ---------------------------------------------------------
pred_lgbm = model_lgbm.predict(X_valid)

rmse, mae, mape, r2, dir_acc = get_detailed_score(y_valid, pred_lgbm)

print("-" * 60)
print(f"⚡ [LightGBM 성적표]")
print(f"   - RMSE: {rmse:.2f}")
print(f"   - MAE : {mae:.2f}")
print(f"   - MAPE: {mape:.2f}%")
print(f"   - r2: {r2:.2f}")
print(f"   - Dir Acc: {dir_acc:.1f}%")
print("-" * 60)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta

# ---------------------------------------------------------
# 1. 재귀적 예측 함수 (LightGBM용)
# ---------------------------------------------------------
def recursive_forecast_lgbm(model, df_initial, features, steps=144):
    future_predictions = []
    future_dates = []
    
    # 히스토리 버퍼
    history_prices = df_initial['Close'].iloc[-100:].tolist()
    last_row = df_initial.iloc[[-1]].copy()
    current_time = last_row.index[0]
    
    for i in range(steps):
        # 예측
        pred_price = model.predict(last_row[features])[0]
        
        future_predictions.append(pred_price)
        next_time = current_time + timedelta(minutes=30)
        future_dates.append(next_time)
        
        # 업데이트 로직
        history_prices.append(pred_price)
        if len(history_prices) > 100: history_prices.pop(0)
        
        s_history = pd.Series(history_prices)
        last_row['MA_5'] = s_history.rolling(5).mean().iloc[-1]
        last_row['MA_48'] = s_history.rolling(48).mean().iloc[-1]
        last_row['Std_20'] = s_history.rolling(20).std().iloc[-1]
        
        delta = s_history.diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean().iloc[-1]
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean().iloc[-1]
        if loss == 0: last_row['RSI'] = 100
        else: last_row['RSI'] = 100 - (100 / (1 + (gain / loss)))
            
        last_row['Close_Lag2'] = last_row['Close_Lag1']
        last_row['Close_Lag1'] = pred_price
        last_row['GPT_Lag1'] = 0.0
        last_row['Hour'] = next_time.hour
        last_row['DayOfWeek'] = next_time.dayofweek
        current_time = next_time
        
    return pd.DataFrame({'Date': future_dates, 'Price': future_predictions})

# ---------------------------------------------------------
# 2. 실행 및 시각화
# ---------------------------------------------------------
print("🔮 LightGBM 미래 3일 예측 시작...")
df_future_lgbm = recursive_forecast_lgbm(model_lgbm, df_ml, features)

plt.figure(figsize=(12, 6))

# 과거 데이터
history_view = df_ml.tail(144) # 최근 3일치
plt.plot(history_view.index, history_view['Close'], label='History', color='black', alpha=0.3)

# LightGBM 예측
plt.plot(df_future_lgbm['Date'], df_future_lgbm['Price'],
         label='LightGBM Forecast', color='dodgerblue', linewidth=2)

plt.axvline(df_ml.index[-1], color='red', linestyle=':', label='Now')
plt.title(f"[{TARGET_ITEM}] LightGBM Future Forecast (3 Days)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
plt.show()

print(f"📊 LightGBM 3일 뒤 예상가: {df_future_lgbm['Price'].iloc[-1]:.0f} G")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# ---------------------------------------------------------
# 1. 점수 계산 (LightGBM)
# ---------------------------------------------------------
print("⚡ [LightGBM 상세 평가]")

# 예측
valid_preds_lgbm = model_lgbm.predict(X_valid)

# 채점
rmse_lgbm = np.sqrt(mean_squared_error(y_valid, valid_preds_lgbm))
mae_lgbm = mean_absolute_error(y_valid, valid_preds_lgbm)

print(f"  - RMSE (평균 오차): ±{rmse_lgbm:.2f} 골드")
print(f"  - MAE (절대 오차): ±{mae_lgbm:.2f} 골드")

# ---------------------------------------------------------
# 2. 학습 곡선 (Learning Curve)
# ---------------------------------------------------------
# LightGBM은 evals_result_ 딕셔너리에 기록이 있습니다.
# 우리가 eval_set=[(X_valid, y_valid)] 하나만 넣었으므로 'valid_0' 키를 사용합니다.
results_lgbm = model_lgbm.evals_result_

# 'rmse' 혹은 'l2' 등의 키로 저장됩니다. (metric 설정에 따라 다름)
# 보통 rmse를 썼다면 'rmse' 키를 찾습니다.
metric_name = list(results_lgbm['valid_0'].keys())[0] # 첫 번째 키(metric) 자동 감지
valid_history = results_lgbm['valid_0'][metric_name]
epochs = len(valid_history)
x_axis = range(0, epochs)

plt.figure(figsize=(12, 5))
# 학습 데이터(Train)를 eval_set에 안 넣었으면 Valid 곡선만 그려집니다.
plt.plot(x_axis, valid_history, label='Valid Error (모의고사)', color='red')
plt.title(f'LightGBM Learning Curve ({metric_name})')
plt.xlabel('Iterations')
plt.ylabel('Error')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# ---------------------------------------------------------
# 3. 실제 vs 예측 비교
# ---------------------------------------------------------
plt.figure(figsize=(12, 5))
plt.plot(y_valid.index, y_valid, label='Actual Price', color='black', alpha=0.5)
plt.plot(y_valid.index, valid_preds_lgbm, label='LightGBM Prediction', color='dodgerblue', linestyle='--')
plt.title(f"[{TARGET_ITEM}] LightGBM Performance Check")
plt.legend()
plt.show()